In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=120,
		# 키로 대화 기록을 저장하는 공간 찾기
    memory_key= "chat_history"
)

template = """

    You are a helpful AI taling to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
		# 해당 프롬프트로 진행하면 메모리에는 저장이 되지만 프롬프트에 포함되지 않는다.
		# prompt= PromptTemplate.from_template({question}),
    prompt= PromptTemplate.from_template(template),
		# 체인을 디버깅(체인의 프롬프트 확인 가능)하기 위한 설정
    verbose=True,
)

chain.predict(question="My name is Justddan")



> Entering new LLMChain chain...
Prompt after formatting:


    You are a helpful AI taling to a human.

    
    Human:My name is Justddan
    You:


> Finished chain.


'Hello Justddan! How can I assist you today?'

In [2]:
chain.predict(question="I lived in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:


    You are a helpful AI taling to a human.

    Human: My name is Justddan
AI: Hello Justddan! How can I assist you today?
    Human:I lived in Seoul
    You:


> Finished chain.


"That's great! Seoul is a vibrant and bustling city. Is there anything specific you would like to know or discuss about Seoul?"

In [3]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:


    You are a helpful AI taling to a human.

    Human: My name is Justddan
AI: Hello Justddan! How can I assist you today?
Human: I lived in Seoul
AI: That's great! Seoul is a vibrant and bustling city. Is there anything specific you would like to know or discuss about Seoul?
    Human:What is my name?
    You:


> Finished chain.


'Your name is Justddan.'